# Ontology Design
This Knowledge-Based System (KBS) uses an ontology to model university course advising.

### Concepts (Classes)
* **Student**: The individual learners in the system.
* **Course**: The academic units available for enrollment.

### Relationships (Properties)
* **requires(Course → Course)**: Prerequisites that must be finished first.
* **completed(Student → Course)**: The record of courses a student has passed.
* **eligibleFor(Student → Course)**: An inferred property; a student is eligible if all prerequisites are completed.

Cell 2: Code - Implementation

In [7]:
class UniversityOntologyKBS:
    def __init__(self):
        self.students = set()
        self.courses = set()
        self.prerequisites = {}  # {course: {prereqs}}
        self.completions = {}    # {student: {completed_courses}}

    def add_student(self, student: str):
        self.students.add(student)
        self.completions[student] = set()

    def add_course(self, course: str):
        self.courses.add(course)
        if course not in self.prerequisites:
            self.prerequisites[course] = set()

    def add_prerequisite(self, course: str, prereq: str):
        if course not in self.courses or prereq not in self.courses:
            raise ValueError(f"Unknown course or prerequisite: {course}, {prereq}")
        self.prerequisites[course].add(prereq)

    def complete_course(self, student: str, course: str):
        if student not in self.students or course not in self.courses:
            raise ValueError(f"Unknown student or course: {student}, {course}")
        self.completions[student].add(course)

    def can_take(self, student: str, course: str) -> (bool, set[str]):
        if student not in self.students or course not in self.courses:
            raise ValueError(f"Unknown student or course: {student}, {course}")
        
        prereqs = self.prerequisites.get(course, set())
        completed = self.completions[student]
        missing = prereqs - completed
        
        return (len(missing) == 0, missing)

    def recommend_courses(self, student: str) -> list[str]:
        if student not in self.students:
            raise ValueError(f"Unknown student: {student}")
            
        completed = self.completions[student]
        recommendations = []
        
        for course in self.courses:
            if course not in completed:
                eligible, _ = self.can_take(student, course)
                if eligible:
                    recommendations.append(course)
        
        return sorted(recommendations)

Cell 3: Code - Dataset

In [8]:
kbs = UniversityOntologyKBS()

# 1. Add 6 Courses
course_list = ["Math101", "Intro_Programming", "Java_SWE3040", "Frameworks_SWE2040", "Database_Systems", "Cloud_Computing"]
for c in course_list: kbs.add_course(c)

# 2. Add 3 Students
student_list = ["Abdinasir", "Mohamed", "Student405"]
for s in student_list: kbs.add_student(s)

# 3. Add 7 Prerequisite links (including 2+ levels depth)
# Depth: Math -> Intro_Programming -> Java -> Frameworks
kbs.add_prerequisite("Intro_Programming", "Math101")
kbs.add_prerequisite("Java_SWE3040", "Intro_Programming")
kbs.add_prerequisite("Frameworks_SWE2040", "Java_SWE3040") # Level 3 chain
kbs.add_prerequisite("Database_Systems", "Intro_Programming")
kbs.add_prerequisite("Cloud_Computing", "Intro_Programming")
kbs.add_prerequisite("Cloud_Computing", "Database_Systems")
kbs.add_prerequisite("Java_SWE3040", "Math101")

# 4. Set Initial Completions
kbs.complete_course("Abdinasir", "Math101")
kbs.complete_course("Abdinasir", "Intro_Programming")
kbs.complete_course("Abdinasir", "Java_SWE3040") # Ready for SWE2040

kbs.complete_course("Mohamed", "Math101") # Partially eligible for Programming
# Student405 has completed nothing

Cell 4: Code - Inference Demo

In [9]:
print("--- Eligibility Checks ---")
# Case 1: Eligible
res1, missing1 = kbs.can_take("Abdinasir", "Frameworks_SWE2040")
print(f"Abdinasir -> SWE2040: {'Eligible' if res1 else 'Not Eligible'}")

# Case 2: Partially Eligible
res2, missing2 = kbs.can_take("Mohamed", "Java_SWE3040")
print(f"Mohamed -> Java_SWE3040: Not Eligible (Missing: {missing2})")

# Case 3: Not Eligible
res3, missing3 = kbs.can_take("Student405", "Frameworks_SWE2040")
print(f"Student405 -> SWE2040: Not Eligible (Missing: {missing3})")

--- Eligibility Checks ---
Abdinasir -> SWE2040: Eligible
Mohamed -> Java_SWE3040: Not Eligible (Missing: {'Intro_Programming'})
Student405 -> SWE2040: Not Eligible (Missing: {'Java_SWE3040'})


Cell 5: Code - Recommendations

In [10]:
print("--- Student Recommendations ---")
for s in student_list:
    print(f"Next courses for {s}: {kbs.recommend_courses(s)}")

--- Student Recommendations ---
Next courses for Abdinasir: ['Database_Systems', 'Frameworks_SWE2040']
Next courses for Mohamed: ['Intro_Programming']
Next courses for Student405: ['Math101']


Cell 6: Code - Unit Tests

In [11]:
import unittest

class TestOntologyKBS(unittest.TestCase):
    def setUp(self):
        """Initializes a fresh KBS for every test to prevent data pollution."""
        self.test_kbs = UniversityOntologyKBS()
        # Add required courses for testing
        for c in ["Math101", "Intro_Prog", "Java_SWE3040"]:
            self.test_kbs.add_course(c)
        self.test_kbs.add_student("TestUser")

    def test_invalid_student(self):
        with self.assertRaises(ValueError):
            self.test_kbs.can_take("Unknown", "Math101")

    def test_no_prereq_eligible(self):
        # Math101 has no prereqs, so TestUser is automatically eligible
        eligible, _ = self.test_kbs.can_take("TestUser", "Math101")
        self.assertTrue(eligible)

    def test_not_eligible_missing_prereq(self):
        self.test_kbs.add_prerequisite("Intro_Prog", "Math101")
        eligible, missing = self.test_kbs.can_take("TestUser", "Intro_Prog")
        self.assertFalse(eligible)
        self.assertIn("Math101", missing)

    def test_recommendation_filtering(self):
        """Fixes the failure: Completed courses must not be recommended."""
        self.test_kbs.complete_course("TestUser", "Math101")
        recs = self.test_kbs.recommend_courses("TestUser")
        self.assertNotIn("Math101", recs)

    def test_prereq_chain_logic(self):
        self.test_kbs.add_prerequisite("Intro_Prog", "Math101")
        self.test_kbs.add_prerequisite("Java_SWE3040", "Intro_Prog")
        # User has nothing, so they shouldn't be able to take the advanced course
        eligible, _ = self.test_kbs.can_take("TestUser", "Java_SWE3040")
        self.assertFalse(eligible)

    def test_successful_recommendation(self):
        self.test_kbs.complete_course("TestUser", "Math101")
        self.test_kbs.add_prerequisite("Intro_Prog", "Math101")
        recs = self.test_kbs.recommend_courses("TestUser")
        self.assertIn("Intro_Prog", recs)

# This line runs the tests inside the notebook
unittest.main(argv=[''], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.007s

OK
